# Tidal signatures search

In Gaia data, can we detect the signatures of tides by looking along the orbit to see excesses of stars of tidal shifts?

Below, I use both a cutout of 4 degrees from gaia using simple absolute cuts and also J+24's sample of bayesian probabilities and make plots of CMD and proper motion space along the orbit.

# Setup

In [ ]:
using Arya, CairoMakie
using LilGuys
import CairoMakie: save
import TOML

In [ ]:
red = COLORS[6];

In [ ]:
import LinearAlgebra: normalize, ×

In [ ]:
# selection parameters

pm_max = 10
θ0 = -39.63
fig_dir = "./figures/"

In [ ]:
obs_props = TOML.parsefile("observed_properties.toml")


In [ ]:
include("../../utils/gaia_filters.jl")

In [ ]:
filt_params = GaiaFilterParams(read_paramfile("processed/simple.toml"))

In [ ]:
distance = obs_props["distance"]

## Loading data

In [ ]:
all_stars = load_gaia_stars("data/gaia_4deg_cen.fits", filt_params, θ = θ0)

all_stars[!, :filt_ruwe] = ruwe_filter(all_stars, filt_params)
all_stars[!, :filt_parallax] = parallax_filter(all_stars, filt_params)
all_stars[!, :filt_qual] = all_stars.filt_ruwe .& all_stars.filt_parallax

all_stars[!, :filt_cmd] = cmd_filter(all_stars, filt_params)

all_stars[!, :filt_pm] = pm_filter(all_stars, filt_params)
all_stars[!, :filt_all] = all_stars.filt_qual .& all_stars.filt_cmd .& all_stars.filt_pm


all_stars

In [ ]:
j24 = load_gaia_stars("processed/j24_sculptor_all.fits", filt_params, θ = θ0)

j24[!, :filt_qual] = j24.F_BEST .== 1.0
j24[!, :filt_cmd] = j24.PSAT_CMD .> 0.2
j24[!, :filt_pm] = j24.PSAT_PM .> 0.2
j24[!, :filt_all] = j24.PSAT .> 0.2

j24

In [ ]:
parentdir = "/astro/dboyea/dwarfs"
modelname = "sculptor/1e6_V31_r3.2/orbit1"

In [ ]:
model_dir = joinpath(parentdir, "analysis", modelname)
orbit_props = TOML.parsefile("$model_dir/orbital_properties.toml")
idx_f = orbit_props["idx_f"]

orbit = LilGuys.read_fits("$model_dir/skyorbit.fits")
orbit[!, :xi], orbit[!, :eta] = LilGuys.to_tangent(orbit.ra, orbit.dec, orbit.ra[idx_f], orbit.dec[idx_f])

orbit = orbit[idx_f-10:idx_f+10, :]

## Plotting functions

In [ ]:
macro savefig(name, fig=nothing)
	if fig === nothing
		fig = esc(:fig)
	end
	
	return quote
		filename = joinpath(fig_dir, $name) * ".pdf"
		save(filename, $fig)
		@info "saved figure to $filename"
	end
end

In [ ]:
function cmd_axis(gs)
	return Axis(gs,
		xlabel = "Bp-Rp",
		ylabel = "G",
		yreversed=true,
		limits = (-0.2, 2, 15, 21),
		xgridvisible=false,
		ygridvisible=false,
	)
end

In [ ]:
function xieta_axis(gs)
	return Axis(gs,
		xlabel=L"$\xi$ / degrees",
		ylabel=L"$\eta$ / degrees",
		aspect=DataAspect(),
		xgridvisible=false, 
		ygridvisible=false,
        xreversed=true,
	)
end

In [ ]:
function pm_axis(gp; dpm=11, kwargs...)
	return Axis(gp, 
		xlabel=L"$\mu_{\alpha*}$ / mas\,yr$^{-1}$",
		ylabel=L"$\mu_\delta$ / mas\,yr$^{-1}$",
		aspect=DataAspect(),
		limits= dpm .* (-1, 1, -1, 1),
		xgridvisible=false,
		ygridvisible=false,
	)
end

# Plots of entire samples

In [ ]:
let
	fig = Figure()
	ax = xieta_axis(fig[1, 1])
    ax.title = "Gaia"
    
	ax.limits = (-4, 4, -4, 4)
	
	df = all_stars[all_stars.filt_cmd .& all_stars.filt_qual, :]
	scatter!(df.xi, df.eta, color=:black, alpha=1, markersize=3, label="quality+parallax+cmd")

	df = all_stars[all_stars.filt_cmd .& all_stars.filt_qual .& all_stars.filt_pm, :]
	scatter!(df.xi, df.eta, color=red, alpha=1, markersize=3, label="+pm")
    Legend(fig[1,2], ax)
    
	@savefig "scl_matched_filter"
	fig
end

In [ ]:
let
	fig = Figure()
	ax = xieta_axis(fig[1, 1])
	ax.limits = (-2, 2, -2, 2)
    ax.title = "J+24"
	
	df = j24[j24.filt_cmd .& j24.filt_qual, :]
	scatter!(df.xi, df.eta, color=:black, alpha=1, markersize=3, label="quality+parallax+cmd")

	df = j24[j24.filt_all, :]
	scatter!(df.xi, df.eta, color=red, alpha=1, markersize=3, label="PSAT")
    Legend(fig[1,2], ax)
    
	fig
end

### Quality cuts

In [ ]:
import LilGuys.Plots as LP

In [ ]:
let
    fig = Figure()
    ax = Axis(fig[1, 1], 
        xlabel = "parallax",
        ylabel = "parallax error",
        limits = (-10, 10, 0, nothing),
        )

    scatter!(all_stars.parallax, all_stars.parallax_error, markersize=1, alpha=0.2, color=:black, label = "all stars")

    scatter!(all_stars.parallax[all_stars.filt_parallax], all_stars.parallax_error[all_stars.filt_parallax], label = "parallax selected stars")

    xs = 1.5 * [-5, 0, 5]
    lines!(xs, 1/3*abs.(xs), label = "3 sigma consist. with zero", color=COLORS[2])

    LP.hide_grid!(ax)

    Legend(fig[1, 2], ax)

    fig
end

In [ ]:
let
    fig = Figure()
    ax = Axis(fig[1, 1], 
        xlabel = "log RUWE", 
        ylabel = "counts",
        limits = (nothing, nothing, 1, 1e5),
        yscale=log10
        )

    bins = -0.1:0.01:1.5

    hist!(log10.(all_stars.ruwe)[all_stars.filt_ruwe], bins=bins, label="RUWE selection")
    stephist!(log10.(all_stars.ruwe), bins=bins, color=:black, label="all stars")

    vlines!(log10(1.3), color=COLORS[3], linestyle=:dash, label="threshold")

    axislegend()

    LP.hide_grid!(ax)
    fig
end

In [ ]:
cmd_x = [filt_params.cmd_cut[1:2:end]; filt_params.cmd_cut[1]]
cmd_y = [filt_params.cmd_cut[2:2:end]; filt_params.cmd_cut[2]];

In [ ]:
let
	fig = Figure()

	ax = cmd_axis(fig[1, 1])

	filt = copy(all_stars.filt_qual)

	scatter!(all_stars.bp_rp[filt], all_stars.G[filt], markersize=1, alpha=0.3, color=:black, label="parallax + quality cuts")

	filt .&= all_stars.filt_cmd .* all_stars.filt_pm
	scatter!(all_stars.bp_rp[filt], all_stars.G[filt], markersize=2, label="+ CMD + PM cuts")
	#lines!(iso.bp_rp, iso.G .+ dm)
	poly!(cmd_x, cmd_y, color=:transparent, strokecolor=COLORS[2], strokewidth=2)

	axislegend(position=:lt, markersize=10)

	@savefig "cmd_polygon"
	fig
end

In [ ]:
let
	fig = Figure()

	ax = cmd_axis(fig[1, 1])
	ax.title = "J24"

	filt = copy(j24.filt_qual)

	scatter!(j24.bp_rp[filt], j24.G[filt], markersize=1, alpha=0.3, color=:black, label="parallax + quality cuts")

	filt .&= j24.filt_all
	scatter!(j24.bp_rp[filt], j24.G[filt], markersize=2, label="+ CMD + PM cuts")
	#lines!(iso.bp_rp, iso.G .+ dm)
	poly!(cmd_x, cmd_y, color=:transparent, strokecolor=COLORS[2], strokewidth=2)

	axislegend(position=:lt, markersize=10)

	@savefig "cmd_polygon"
	fig
end

In [ ]:
let
	fig = Figure()
    ax = pm_axis(fig[1, 1], dpm=11)
    ax.title = "Gaia stars within 4 degrees of Scl"
    
	scatter!(all_stars.pmra, all_stars.pmdec, markersize=1, alpha=0.2, color=:black, )
	scatter!([NaN], [NaN], markersize=5, alpha=0.2, color=:black, label = "all stars in Gaia")
    
	filt = all_stars.filt_pm 

	scatter!(all_stars.pmra[filt], all_stars.pmdec[filt], markersize=1, color=COLORS[1])
	scatter!([NaN], [NaN], markersize=5, color=COLORS[1], label = "PM selected stars")

    arc!(Point2f(filt_params.pmra, filt_params.pmdec), filt_params.dpm, -π, π, color=COLORS[2], label = "PM cut")

    leg = Legend(fig[1, 2], ax, markersize=15)
    
	fig
end

In [ ]:
let
	fig = Figure()
    ax = pm_axis(fig[1, 1], dpm=11)
    
	scatter!(j24.pmra, j24.pmdec, 
        markersize=2, alpha=0.2, color=:black, 
    )
    scatter!([NaN], [NaN], markersize=5, alpha=0.2, color=:black, label = "J+24 all stars")

	filt = j24.filt_pm 
	scatter!(j24.pmra[filt], j24.pmdec[filt], markersize=2, )
	scatter!([NaN], [NaN], markersize=5, color=COLORS[1], label="J+24 PM PSAT > 0.2")
    arc!(Point2f(filt_params.pmra, filt_params.pmdec), filt_params.dpm, -π, π, color=COLORS[2], label = "PM cut")

    Legend(fig[1, 2], ax)
	fig
end

### Understanding the background

In [ ]:
let
    fig = Figure()
    ax = xieta_axis(fig[1,1])

    filt = all_stars.filt_qual .& .! all_stars.filt_cmd .& .! all_stars.filt_pm

    scatter!(all_stars.xi[filt], all_stars.eta[filt], color=:black, markersize=1)

    ax.title = "not CMD, not PM but passing parallax and ruwe"
    fig
end

In [ ]:
let
    fig = Figure()
    ax = cmd_axis(fig[1,1])

    filt = all_stars.filt_qual .& .! all_stars.filt_pm

    scatter!(all_stars.bp_rp[filt], all_stars.G[filt], color=:black, markersize=1)

    ax.title = "not PM but passing parallax and ruwe"
    fig
end

In [ ]:
let
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel = L"total parallax error (mas\,yr$^{-1}$)",
        ylabel = "count",
        title = "J+24, PSAT > 0.2",
        )

    df = j24[j24.filt_all, :]
    x = df.pmra_error .⊕ df.pmdec_error
    stephist!(x)
    
    vlines!(median(x), color=COLORS[2])
    text!(median(x), 0, text="median", color=COLORS[2], rotation=π/2)

    println(median(x))
    fig
end

In [ ]:
let
    fig = Figure()
    ax = Axis(fig[1, 1],
        xlabel = L"total parallax error (mas\,yr$^{-1}$)",
        ylabel = "count",
        title = "all ",
        )

    df = all_stars[all_stars.filt_qual, :]
    x = df.pmra_error .⊕ df.pmdec_error
    x = x[isfinite.(x)]
    stephist!(x)
    
    vlines!(median(x), color=COLORS[2])
    text!(median(x), 0, text="median", color=COLORS[2], rotation=π/2)

    println(median(x))
    fig
end

In [ ]:
let
    fig = Figure()
    ax = pm_axis(fig[1,1], dpm=20)

    filt = all_stars.filt_qual .& .! all_stars.filt_cmd

    scatter!(all_stars.pmra[filt], all_stars.pmdec[filt], color=:black, markersize=1)

    scatter!(filt_params.pmra, filt_params.pmdec, color=red)
    ax.title = "not CMD but passing parallax and ruwe"
    fig
end

In [ ]:
let
    fig = Figure()
    ax = xieta_axis(fig[1,1])
    ax.title = "J+24, PSAT < 0.2"

    filt = j24.PSAT .< 0.2

    scatter!(j24.xi[filt], j24.eta[filt], color=:black, markersize=2)

    fig
end

In [ ]:
let
    fig = Figure()
    ax = cmd_axis(fig[1,1])
    ax.title = "J+24, PSAT < 0.2"

    filt = j24.PSAT .< 0.2

    scatter!(j24.bp_rp[filt], j24.G[filt], color=:black, markersize=2)

    fig
end

In [ ]:
let
    fig = Figure()
    ax = pm_axis(fig[1,1])
    ax.title = "J+24, PSAT < 0.2"

    filt = j24.PSAT .< 0.2

    scatter!(j24.pmra[filt], j24.pmdec[filt], color=:black, markersize=2)
    scatter!(filt_params.pmra, filt_params.pmdec, color=red)

    fig
end

# Cutouts

## Defining the cutouts

Here, we begin to take cutouts along the orbit (more specifically, along the proper motion vector in the GSR frame as this is model-independent)

In [ ]:
orbit_vector = [sind(θ0), cosd(θ0)]
bg_vector = ([orbit_vector; 0] × [0, 0, 1])[1:2]

In [ ]:
r_circ_cut = 0.25

## Utility functions

In [ ]:
function select_region_circle(allstars, centre; radius=0.5)
	x_cen, y_cen = centre

	r = @. sqrt((allstars.xi - x_cen)^2 + (allstars.eta - y_cen)^2)

	filt = r .< radius

	return filt
end

In [ ]:
function select_region_rect(all_stars, low, high; radius=1, θ=0)

    x = @. cosd(θ) * all_stars.xi_p + sind(θ) * all_stars.eta_p
    y = @. sind(θ) * all_stars.xi_p - cosd(θ) * all_stars.eta_p
    
	filt = abs.(y) .<= radius

    filt .&= low .<= x .< high
    
	return filt
end

In [ ]:
function plot_cmd_members!(ax, all_stars, filters; labels=nothing)    
    if labels === nothing
        labels = ["" for _ in filters]
    end

    for i in 1:length(filters)
        df = all_stars[filters[i], :]
        plot_cmd!(ax, df, label=labels[i]; styles[i]...)
	end
    
	N = sum(filters[end])
    return N
end

In [ ]:

styles = [ 
    (; markersize=5, color=COLORS[1]), 
    (; color=COLORS[2], markersize=7),
    (; color=COLORS[3], markersize=10),
]


In [ ]:
function plot_tangent_members!(ax, all_stars, filters; labels=false, background=false)
    if labels === nothing
        labels = ["" for _ in filters]
    end

    s = []
    for i in 1:length(filters)
        df = all_stars[filters[i], :]
        p = scatter!(ax, df.xi, df.eta, label=labels[i]; styles[i]...)
        push!(s, p)
	end

    return s
end

In [ ]:
function plot_pm_members!(ax, all_stars, filters; labels=false)
    if labels === nothing
        labels = ["" for _ in filters]
    end


    for i in 1:length(filters)
        df = all_stars[filters[i], :]
        scatter!(ax, df.pmra, df.pmdec, label=labels[i]; styles[i]...)
	end

    df = all_stars[filters[end], :]
	x, y, xe, ye = get_mean_pm(df)

	scatter!(ax, x, y, #xerr=[pmra_cen_err], yerr=[pmdec_cen_err], 
	color=red, label="mean", markersize=15)
    errscatter!([x], [y], xerr=[xe], yerr=[ye], 
	color=red, label="mean")

end

In [ ]:
function get_mean_pm(df)
	pmra_cen = lguys.mean(df.pmra, (df.pmra_error .^ -2))
	pmra_cen_err = sqrt(1 / sum(df.pmra_error .^ -2))
	pmdec_cen = lguys.mean(df.pmdec, (df.pmdec_error .^ -2))
	pmdec_cen_err = sqrt(1 / sum(df.pmdec_error .^ -2))

	return pmra_cen, pmdec_cen, pmra_cen_err, pmdec_cen_err
end

In [ ]:
function plot_cmd!(ax, df; kwargs...)
	scatter!(ax, df.bp_rp, df.G, alpha=1; kwargs...)
end

# Animations

To understand if there are substantial numbers of stars which could be associated with sculptor

In [ ]:
using Printf

In [ ]:
function compare_regions_animation(all_stars, selection_filters; 
        legend=true, r_max=4, Nf=20, θ=0, filename="compare_regions_animation.gif"
    )
    dr = 0.5
    rs_test = LinRange(-r_max, r_max-dr, Nf)
    
        
    labels = first.(selection_filters)
    
    # Create observables to update during the animation
    filt_r_obs = select_region_rect(all_stars, rs_test[1], rs_test[1]+dr, θ=θ)
    filts_obs = [filt .& filt_r_obs for (_, filt) in selection_filters]
    
    # Create the figure layout
    fig = Figure(size = (800, 600))


    ax = xieta_axis(fig[1, 1])
    ax.limits = r_max .* (-1, 1, -1, 1)
    filt = selection_filters[end].second
    scatter!(ax, all_stars.xi[filt], all_stars.eta[filt]; color=COLORS[length(selection_filters)], markersize=1)
    lines!(ax, orbit.xi, orbit.eta, color=COLORS[1])
    xieta_scatters = plot_tangent_members!(ax, all_stars, filts_obs, labels=labels)

    
    ax_cmd = cmd_axis(fig[1, 2])
    Ns = plot_cmd_members!(ax_cmd, all_stars, filts_obs, labels=labels)
    text!(ax_cmd, 0.05, 0.95, text="$Ns", space=:relative, align=(:left, :top))

    ax_cmd.title = @sprintf "R = %0.2f" rs_test[1] + dr/2
    
    ax_pm = pm_axis(fig[2, 1])
    plot_pm_members!(ax_pm, all_stars, filts_obs, labels=labels)


    if legend && (labels !== nothing)
        Legend(fig[2, 2], fig.content[1], tellwidth=false)
    end

    
    # Animation: Update observable and capture frames
    record(fig, joinpath(fig_dir, filename), 1:Nf, framerate=2) do frame
        for p in xieta_scatters
            delete!(ax, p)
        end
        
        empty!(ax_cmd)
        empty!(ax_pm)
        
        filt_r_obs = select_region_rect(all_stars, rs_test[frame], rs_test[frame]+dr, θ=θ)
        filts_obs = [filt .& filt_r_obs for (_, filt) in selection_filters]

        ax_cmd.title = @sprintf "R = %0.2f" rs_test[frame] + dr/2


        xieta_scatters = plot_tangent_members!(ax, all_stars, filts_obs, labels=labels)
        Ns = plot_cmd_members!(ax_cmd, all_stars, filts_obs, labels=labels)
        plot_pm_members!(ax_pm, all_stars, filts_obs, labels=labels)
        
        text!(ax_cmd, 0.05, 0.95, text="$Ns stars", space=:relative, align=(:left, :top))
    end

    fig
end

## J24 regions

In [ ]:
filts_j24 = [
    "quality"=>j24.filt_qual, 
    "psat"=>j24.filt_all
]

In [ ]:
compare_regions_animation(j24, filts_j24, r_max=2, filename = "compare_regions_j24.gif"
)

## All stars

In [ ]:
filts_all_stars = [
    "parallax + RUWE" => all_stars.filt_qual, 
    "+ in CMD polygon" => all_stars.filt_qual .& all_stars.filt_cmd]


In [ ]:
compare_regions_animation(all_stars, filts_all_stars,
)

In [ ]:
compare_regions_animation(all_stars, filts_all_stars,
    θ = 90,
    filename = "compare_regions_perp.gif"
)

## All stars + PM

In [ ]:
filts_all_stars = [
    "parallax + RUWE" => all_stars.filt_qual, 
    "+ in CMD polygon + PM" => all_stars.filt_qual .& all_stars.filt_cmd .& all_stars.filt_pm]


In [ ]:
compare_regions_animation(all_stars, filts_all_stars, 
    filename = "compare_regions_pm.gif"
)

In [ ]:
compare_regions_animation(all_stars, filts_all_stars, 
    θ = 90,
    filename = "compare_regions_pm_perp.gif"
)

# Binned properties along orbit

In [ ]:
using DataFrames

In [ ]:
function compute_pm_stats(all_stars::DataFrame; Nb=20, r_max=4, θ=0, dr=0.5)
    # Define the radial bins
    r_bins = LinRange(-r_max, r_max - dr, Nb)
   

    # Initialize arrays to store the results
    pmra_means = Float64[]
    pmdec_means = Float64[]
    pmra_means_err = Float64[]
    pmdec_means_err = Float64[]
    pm_counts = Int[]

    # Iterate through the radial bins
    for i in 1:Nb
        # Select stars in the current bin
        filt = select_region_rect(all_stars, r_bins[i], r_bins[i] + dr, θ = θ)

        println(i, "\t", round(r_bins[i], digits=2), "\t", sum(filt))
        # Calculate the mean and errors for proper motions
        x, y, xe, ye = get_mean_pm(all_stars[filt, :])

        # Append the results to the respective arrays
        push!(pmra_means, x)
        push!(pmdec_means, y)
        push!(pmra_means_err, xe)
        push!(pmdec_means_err, ye)
        push!(pm_counts, sum(filt))
    end

    # Create a DataFrame with the computed statistics
    df = DataFrame(
        x_bin_start = r_bins,
        x_bin_end = r_bins .+ dr,
        x = r_bins .+ dr / 2,
        pmra_mean = pmra_means,
        pmdec_mean = pmdec_means,
        pmra_mean_err = pmra_means_err,
        pmdec_mean_err = pmdec_means_err,
        pm_count = pm_counts
    )

    return df
end

In [ ]:
all_stars[!, :filt_pm_fancy] = pm_filter(all_stars, filt_params.pmra, filt_params.pmdec, 0.1, 3);

In [ ]:
df_means = compute_pm_stats(all_stars[all_stars.filt_all, :]);

In [ ]:
df_means_fancy = compute_pm_stats(all_stars[all_stars.filt_cmd .& all_stars.filt_qual .& all_stars.filt_pm_fancy, :]);

In [ ]:

df_nopm = compute_pm_stats(all_stars[all_stars.filt_cmd .& all_stars.filt_qual, :])
df_bg = compute_pm_stats(all_stars[all_stars.filt_qual, :])


df_means_eta = compute_pm_stats(all_stars[all_stars.filt_all, :], θ=90)
df_means_fancy_eta = compute_pm_stats(all_stars[all_stars.filt_cmd .& all_stars.filt_qual .& all_stars.filt_pm_fancy, :], θ=90)

df_nopm_eta = compute_pm_stats(all_stars[all_stars.filt_cmd .& all_stars.filt_qual, :], θ=90)

In [ ]:
let

    fig = Figure(size=(800, 400))
	ax = Axis(fig[1, 1],
		xlabel=L"$\xi'$ / degrees", 
		ylabel=L"$\mu_{\alpha*}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means.x, df_means.pmra_mean, yerr=df_means.pmra_mean_err, label="pm filter")
	errscatter!(df_nopm.x, df_nopm.pmra_mean, yerr=df_nopm.pmra_mean_err, label="no pm filter")


    hlines!(
        median(all_stars.pmra[all_stars.filt_qual .& .! all_stars.filt_cmd .& .! all_stars.filt_pm]), 
        label="nonmember median",
        color=COLORS[2],
    )


    Axis(fig[1, 2],
		xlabel=L"$\xi'$ / degrees", 
		ylabel=L"$\mu_{\delta}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means.x, df_means.pmdec_mean, yerr=df_means.pmdec_mean_err)
	errscatter!(df_nopm.x, df_nopm.pmdec_mean, yerr=df_nopm.pmdec_mean_err)
    hlines!(
        median(all_stars.pmdec[all_stars.filt_qual .& .! all_stars.filt_cmd .& .! all_stars.filt_pm]), 
        label="nonmember mean",
    color=COLORS[2],
    )
    

    Legend(fig[1, 3], ax)
    
    fig
end

In [ ]:
let

    fig = Figure(size=(800, 400))
	ax = Axis(fig[1, 1],
		xlabel=L"$\xi'$ / degrees", 
		ylabel=L"$\mu_{\alpha*}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means.x, df_means.pmra_mean, yerr=df_means.pmra_mean_err, label="pm filter")



    Axis(fig[1, 2],
		xlabel=L"$\xi'$ / degrees", 
		ylabel=L"$\mu_{\delta}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means.x, df_means.pmdec_mean, yerr=df_means.pmdec_mean_err)

    fig
end

In [ ]:
let

    fig = Figure(size=(800, 400))
	ax = Axis(fig[1, 1],
		xlabel="r / degrees", 
		ylabel="log N in region",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	scatter!(df_means.x, log10.(df_means.pm_count), label="along orbit, cmd + pm")
	scatter!(df_means_eta.x, log10.(df_means_eta.pm_count), label="⟂ orbit, cmd + pm")
    df = df_nopm
	scatter!(df.x, log10.(df.pm_count), label="along orbit, no pm")
    df = df_bg
	scatter!(df.x, log10.(df.pm_count), label="along orbit, only parallax + ruwe")

    Legend(fig[1, 2], ax)
    fig
end

In [ ]:
let

    fig = Figure(size=(800, 400))
	ax = Axis(fig[1, 1],
		xlabel=L"$\eta'$ / degrees", 
		ylabel=L"$\mu_{\alpha*}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

    df = df_means_eta
    errscatter!(df.x, df.pmra_mean, yerr=df.pmra_mean_err, label="pm filter")



    Axis(fig[1, 2],
		xlabel=L"$\xi'$ / degrees", 
		ylabel=L"$\mu_{\delta}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df.x, df.pmdec_mean, yerr=df.pmdec_mean_err)

    fig
end

In [ ]:
let

    fig = Figure(size=(800, 400))
	ax = Axis(fig[1, 1],
		xlabel=L"$\xi'$ / degrees", 
		ylabel=L"$\mu_{\alpha*}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means_fancy.x, df_means_fancy.pmra_mean, yerr=df_means_fancy.pmra_mean_err, label="pm filter")



    Axis(fig[1, 2],
		xlabel=L"$\xi'$ / degrees", 
		ylabel=L"$\mu_{\delta}$ / mas\,yr$^{-1}$",
		#limits=(-1.1, 1.1, 0, 0.3)
	)

	errscatter!(df_means_fancy.x, df_means_fancy.pmdec_mean, yerr=df_means_fancy.pmdec_mean_err)

    fig
end